In [ ]:
import numpy as np
from luq.luq import *
from scipy.stats import norm, beta # for data-generating distributions
from scipy.stats import gaussian_kde as GKDE
from scipy.integrate import quadrature # for calculating 1-D TV metrics
from tabulate import tabulate # for presenting results for iterative approach
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})
import matplotlib.tri as tri
import ipywidgets as wd

# color palette
c = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']

# setup fontsizes for plots
plt_params = {'legend.fontsize': 14,
          'figure.figsize': (10,8), #(6.4, 4.8),
         'axes.labelsize': 16,
         'axes.titlesize': 16,
         'xtick.labelsize': 14,
         'ytick.labelsize': 14}
plt.rcParams.update(plt_params)

# Details on Data-Generating Samples

## Generating Observed Data

Data was created in the notebook/script labeled 'generating_data' found with this notebook. The model is the 2-D wave equation $$\dfrac{\partial^2 u}{\partial t^2}=\dfrac{\partial^2 u}{\partial x^2}+\dfrac{\partial^2 u}{\partial y^2}, \quad \left(x,y\right)\in (0,5)^2$$ with $u=u(x,y,t)$ and boundary conditions $u(0,y,t)=u(x,0,t)=u(5,y,t)=u(x,5,t)=0$. The problem is to model a water droplet at location $(a,b)$ given by $$u(x,y,0)=0.2\text{exp}\left(-10\left(\left(x-a\right)^2+\left(y-b\right)^2\right)\right)$$ where the location $(a,b)$ has some unknown distribution creating uncertain model outputs, and the goal is to quantify the uncertainty in the droplet locations using observed uncertainties in model outputs. The droplet locations are given a data-generating distribution  described by independent distributions for $a$ and $b$ where $a$ is taken to be a Beta(2,5) distribution scaled and shifted to be on the interval $[1,2]$ and $b$ is taken to be a $N(2.5,0.5)$ normal distribution. The data is generated by creating 200 i.i.d. samples from this data-generating distributions and solving the model using a standard centered finite difference scheme on a 101x101 regular uniformly-spaced mesh on $[0,5]^2$ using 0.005 sized time-steps. The data is then extracted at spatial points $(0.5i,0.5j)$ for $i,j=1,\dots,9$, i.e., on a 9x9 sub-grid of the original 101x101 grid, and at times $t=0.5,1.0,1.5,\dots,7.0$. We simulate noise on this observed data using a normal distribution centered at 0 with standard deviation 2.5E-3 in both spatial directions. 

In [ ]:
# parameter samples for construction of pi_obs

num_obs_samples = 200
np.random.seed(12345678)
params_obs = np.vstack([2 * np.random.beta(a=2, b=5, size=num_obs_samples) + 1,
                         np.random.normal(loc=2.5, scale=0.5, size=num_obs_samples)]) # unknown data-generated parameters corresponding to observed samples

obs = np.load('dg_samples/obs', allow_pickle=True) # noisy observed samples

## Generating Predicted Data

The predicted data is generated using 1000 i.i.d. uniform samples of $[0,5]^2$ and solving the model as described above, but extracting the data on a finer 49x49 grid at spatial points $(0.1i,0.1j)$ for $i,j=1,\dots,49$.

In [ ]:
# parameter samples of pi_init

num_samples = int(1E3)

np.random.seed(123456)
params = np.random.uniform(low=0.0,high=5.0,size=(2,num_samples)) # uniformly distributed parameter samples

In [ ]:
# finite-difference scheme

# defining model solve function
dx = 0.05
dy = 0.05
dt = 0.005 # satifies CFL condition

xn = np.linspace(0,5.0,101) # 101 = length in x / dx
ym = np.linspace(0,5.0,101)
tk = np.linspace(0,7.0,1401) # 1401 = length in t / dt

# defining model solve on 101x101 uniform mesh of [0,5]^2 for t = 0 to t = 7 with dt = 0.005
def M(a,b):
    # initializing the model solution
    # using Dirichlet boundary conditions,so initializing with zeros means boundary values are set
    u = np.zeros((101,101,1401))
    
    # iterate through times; t here is equivalent to time and time index
    for t in range(1401):
        
        # if t = 0, use initial condition modeling wave droplet
        if t == 0:
            mesh = np.meshgrid(xn[1:-1],ym[1:-1])
            u[1:-1,1:-1,t] = 0.2*np.exp(-10*((mesh[0].T-a)**2+(mesh[1].T-b)**2))
        
        # else solve model using finite-difference scheme
        else:
            u[1:-1,1:-1,t] = 2 * u[1:-1,1:-1,t-1] - u[1:-1,1:-1,max(0,t-2)] \
                + dt**2 / dx**2 * (u[2:,1:-1,t-1] - 2 * u[1:-1,1:-1,t-1] + u[:-2,1:-1,t-1]) \
                + dt**2 / dy**2 * (u[1:-1,2:,t-1] - 2 * u[1:-1,1:-1,t-1] + u[1:-1,:-2,t-1])
    return u

# indexing for extracting data on different grid sizes

# indexing function for flattening data
def idx_at(x,y):
    idx = []
    idx.append((x / dx).astype(int))
    idx.append((y / dy).astype(int))
    return idx

# using indexing function to extract data on uniformly-spaced mesh given by delta
def create_idx(delta):
    N = (5-delta)/delta 
    # note: only delta such that (5-delta)/delta is int can be used (or does not change value when cast as int) 
    # any other delta value requires extrapolation
    pts = np.linspace(delta,5-delta,int(N))
    grid_pts = np.meshgrid(pts,pts)
    idx = idx_at(grid_pts[0],grid_pts[1])
    return [idx[0].flatten(), idx[1].flatten()]

# creating grid of size grid_size x grid_size for data coordinates
def create_coordinates(grid_size):
    delta = 5 / (grid_size + 1)
    X, Y = np.meshgrid(range(grid_size),range(grid_size))
    X = X / grid_size * (5 - delta) + delta
    Y = Y / grid_size * (5 - delta) + delta
    return np.vstack([X.flatten(), Y.flatten()]).T

In [ ]:
# predicted data samples on 49x49 grid, 0.1 mesh size

pred = np.zeros((num_samples,49**2,14))
idx = create_idx(0.1)
for i in range(num_samples):
    tmp = M(params[0,i], params[1,i])
    pred[i,:,:] = tmp[idx[0],idx[1],100::100]
    print(f'Predicted sample {i} done.')

# Filtering Noisy Spatial Data

The noisy observed data is filtered by centereing the data with 0 mean in all directions, then fitting a weighted sum of 1-7 Gaussians iteratively, starting with 1 Gaussian and increasing the number of Gaussian's used until either 7 Gaussians are used or a relative error is within a tolerance of $10^{-4}$. This is all done within the $\texttt{LUQ}$ package that uses the new supplemental $\texttt{RBFFit}$ package within $\texttt{LUQ}$.

In [ ]:
# # code used to filter data on 9x9 grid of spatial locations

# # instantiate LUQ at each time step
# learn = [[None]]*4 # excluding first 4 time steps

# for i in range(4,obs.shape[-1]): # observed data has shape (num_samples,num_spatial_locations,num_time_steps)
#     pred_data = pred[:,:,i]
#     obs_data = obs[:,:,i]
#     learn.append(LUQ(predicted_data=pred_data,
#                       observed_data=obs_data))

# # filtering observed data at each time step using sum of Gaussians

# data_coordinates = create_coordinates(9)

# np.random.seed(44444)
# import pickle

# # iterate over time steps
# for i in range(2,obs.shape[-1]):
#     learn[i].filter_data(filter_method='rbfs',
#                           filtered_data_coordinates=data_coordinates,
#                           num_rbf_list=range(1,8),
#                           initializer='kmeans',
#                           max_opt_count=10,
#                           filter_predictions=False,
#                           verbose=True)
#     fn = 'post_filtering/params_t' + str(i)
#     with open(fn, 'wb') as pf:
#         pickle.dump(learn[i].filtered_obs_params, pf)

# loading pre-computed filtered data as luq instance
import pickle

learn = [[None]]*4 # excluding first 4 time steps
for i in range(4,obs.shape[-1]):
    fn = 'post_filtering/params_t' + str(i)
    with open(fn, 'rb') as pf:
        filtered_obs_params = pickle.load(pf)
    # re-instantiating LUQ
    pred_data = pred[:,:,i]
    obs_data = obs[:,:,i]
    learn.append(LUQ(pred_data,
                     obs_data))
    learn[i].filtered_obs = obs_data # needed for shape of filtered_obs when re-evaluating using new_data_coordinates
    learn[i].filtered_obs_params = filtered_obs_params # fitted parameters

    # updating obs_filtering_params info; only need filter_method, remove_trend, add_poly, and poly_deg for recreating data
    learn[i].info['obs_filtering_params'] = {'filter_method': 'rbfs',
                                          'num_rbf_list': range(1,8),
                                          'remove_trend': False,
                                          'add_poly': False,
                                          'poly_deg': None,
                                          'initializer': 'kmeans',
                                          'max_opt_count': 10,
                                          'tol': 0.0001}

    # creating data coordinates where observed data was created
    learn[i].observed_data_coordinates = create_coordinates(9)

## Matching Dimensionality of Predicted and Observed Data

We need the observed and predicted data to exist in the same dimensional space. At this point, the observed data is in a $9^2$ dimensional space while the predicted data is in a $49^2$ dimensional space. Using the fitted function parameters learned with the above filtering step, we re-evaluate the observed data on the finer 49x49 grid to match dimensionality of the observed and predicted data spaces.

In [ ]:
# re-evaluating filtered observations on same grid as predictions

data_coordinates = create_coordinates(49)

for t in range(4,obs.shape[-1]):
    learn[t].filtered_predictions = pred[:,:,t]
    learn[t].new_data_coordinates(data_coordinates, 
                                  recalc_pred=False)
    
    print(f'Predicted data shape: {learn[t].predicted_data.shape}')
    print(f'Filtered observed data shape: {learn[t].filtered_obs.shape}')
    print()

# Learning QoI Map and Transforming Data to Learned QoI Samples

In [ ]:
# learning 2 QoI's from data using kernel pca and transforming the data into QoI samples

for t in range(4,obs.shape[-1]):
    learn[t].learn_qois_and_transform(num_qoi=2)

# Computing DCI Solution Iteratively

## Iterative Algorithm Details

The iterative approach iterates through each time step, calculates the corresponding samples of $$r\left(a,b\right):=\dfrac{\pi_{\text{obs}}\left(Q\left(a,b\right)\right)}{\pi_{\text{pred}}\left(Q\left(a,b\right)\right)}$$ where $Q$ represents the QoI map on the parameters $a$ and $b$. This is done first using both QoI components, but if the DCI diagnostic is not within the specified error tolerance, then the procedure is restarted using one less QoI component until no components are left or until error tolerance is reached. There is then an optional 'quality' check that can be performed to decide whether the iteration should be included or not. In the code below, the algorithm without the quality check is labeled as alg=0 while the algorithm with the quality check is labeled as alg=1.

In [ ]:
# generate kernel density estimates on new QoI and calculate new weights

num_qoi = 2
tol = 0.095
r_vals = []
r_means = []

times_used = []
num_qoi_used = []

for alg in [0,1]: # computing solution without and with optional quality check
    r_vals.append([])
    r_means.append([])
    times_used.append([])
    num_qoi_used.append([])
    for t in range(4,obs.shape[-1]): # excluding first time steps due to too low signal-to-noise ratio
        for n in reversed(range(1,num_qoi+1)): # iterate over number of QoI from max 2 down to 0
            if len(r_vals[alg]) == 0:
                weights = np.ones(learn[t].predict_maps[0][:,:n].shape[0]) # weights of all ones for initial iteration
            else:
                weights = np.prod(r_vals[alg], axis=0) # weights from previous time step for non-initial iterations
            pi_pred = GKDE(learn[t].predict_maps[0][:,:n].T, weights=weights) # computing new predicted density on learned QoI
            pi_obs = GKDE(learn[t].obs_maps[0][:,:n].T) # computing observed density
            # computing new r-values associated with learned QoI
            r_vals[alg].append(
                np.divide(
                    pi_obs(
                        learn[t].predict_maps[0][:,:n].T), 
                    pi_pred(
                        learn[t].predict_maps[0][:,:n].T))) 
            r = weights*r_vals[alg][-1] # new proposed r-samples
            r_means[alg].append(np.mean(r))
            if np.abs(1-r_means[alg][-1]) <= tol: # checking DCI diagnostic
                if alg == 0 or np.mean(r**2) > np.mean(weights**2): # alg=0 will use data; alg=1 will only use data if optional quality check holds
                    times_used[alg].append(0.5*(t+1))               
                    num_qoi_used[alg].append(n)
                else:
                    r_vals[alg].pop()
                    r_means[alg].pop()        
                break # breaks if DCI diagnostic is within tolerance; otherwise, r is re-computed using fewer QoI components with previous results removed
            else:
                r_vals[alg].pop() 
                r_means[alg].pop()

# presenting results for both algorithms
tables = []
for alg in [0,1]:
    tables.append([])
    for i in range(len(r_vals[alg])):
        r = np.prod(r_vals[alg][:i+1],axis=0) # r samples at each iteration
        tables[alg].append([i+1,times_used[alg][i],num_qoi_used[alg][i],r_means[alg][i],np.mean(r**2)])
    print(f'Results for algorithm {alg}:')
    print(tabulate(tables[alg], headers=['iteration','time','number of QoIs used','E_init(r)','E_update(r)']))
    print()

# Results

## Visualizing Updated Densities

In [ ]:
# defining uniform distribution for initial density 
def unif_dist(x, p_range):
    y = np.zeros(x.shape)
    val = 1.0/(p_range[1] - p_range[0])
    for i, xi in enumerate(x):
        if xi < p_range[0] or xi >  p_range[1]:
            y[i] = 0
        else:
            y[i] = val
    return y

# calculating eact data-generating marginals
exact_param_marginals = [lambda x : beta.pdf((x-1)/2,2,5)/2,
                         lambda x : norm.pdf(x,2.5,0.5)]

# calculating exact data-generating joint
np.random.seed(1234) # for reproducibility
params_graphing = np.random.uniform(low=0.0,high=5.0,size=(2,10000)) # large number of uniform parameter samples for graphing

exact_dg = lambda x, y : exact_param_marginals[0](x)*exact_param_marginals[1](y)
exact_dg = exact_dg(params_graphing[0,:],params_graphing[1,:])
kde_dg = GKDE(params_obs)(params_graphing)

# KDEs of true marginals
kde_param_marginals = []
for i in range(params.shape[0]):
        kde_param_marginals.append(GKDE(params_obs[i,:]))

In [ ]:
# constructing and plotting updated marginals

x_min = 0.0
x_max = 5.0
delta = 0.25*(x_max - x_min)
x = np.linspace(x_min-delta, x_max+delta, 100)
param_labels = [r'$a$', r'$b$']
param_marginals = []
param_str = ['a', 'b']

for alg in [0,1]:
    param_marginals.append([])
    for i in range(params.shape[0]):
        plt.figure()
        plt.plot(x, unif_dist(x,[0.0,5.0]), label='Initial', linewidth=2, c=c[0])
        param_marginals[alg].append(GKDE(params[i,:], weights=np.prod(r_vals[alg], axis=0)))
        mar = param_marginals[alg][i](x)
        plt.plot(x, mar, label = 'Updated', linewidth=4, linestyle='dashed', c=c[1])
        plt.plot(x, exact_param_marginals[i](x), label='Data-generating', linewidth=4, linestyle='dotted', c=c[2])
        plt.title('Densities for parameter '+param_labels[i]+f' using algorithm {alg+1}')
        plt.xlabel(param_labels[i])
        plt.legend()
        plt.tight_layout()
#         if alg == 0:
#             fn = 'plots/wave_marginal_' + param_str[i] + '_4.png'
#         else:
#             fn = 'plots/wave_marginal_' + param_str[i] + '_4_r2.png'
#         plt.savefig(fn, bbox_inches='tight')

## Visualizing Iterative Updates

In [ ]:
# plotting updated marginals at each iteration

param_marginals = []

for alg in [0,1]:
    param_marginals.append([[],[]])
    for i in range(params.shape[0]):
        plt.figure()
        plt.plot(x, unif_dist(x,[0.0,5.0]), label='Initial', linewidth=2, c=c[2])
        plt.plot(x, exact_param_marginals[i](x), label='Data-generating', linewidth=2, linestyle='dotted', c=c[1])
        # plt.plot(x, kde_param_marginals[i](x), label='KDE', linewidth=4, c=c[1])
        for j in range(len(r_vals[alg])):
            param_marginals[alg][i].append(GKDE(params[i,:], weights=np.prod(r_vals[alg][:j+1],axis=0)))
            if j == len(r_vals)-1:
                plt.plot(x, param_marginals[alg][i][j](x), label=f'final iteration', linewidth=2, c=c[0], alpha=(j+1)/len(r_vals[alg]))
            else:
                plt.plot(x, param_marginals[alg][i][j](x), label=f'iteration {j+1}', linewidth=2, linestyle='dashed', c=c[0], alpha=(j+1)/len(r_vals[alg]))
        plt.title('Updated densities for parameter '+param_labels[i]+f' using algorithm {alg+1}')
        plt.xlabel(param_labels[i])
        plt.legend()
        plt.tight_layout()
#         if alg == 0:
#             fn = 'plots/wave_marginal_' + param_str[i] + '_iter.png'
#         else:
#             fn = 'plots/wave_marginal_' + param_str[i] + '_iter_r2.png'
#         plt.savefig(fn, bbox_inches='tight')

In [ ]:
# color plot of updated density

pi_updates = []
for alg in [0,1]:
    pi_updates.append(GKDE(params, weights=np.prod(r_vals[alg], axis=0))(params_graphing))
    plt.figure()
    plt.scatter(params_graphing[0,:], params_graphing[1,:], c=pi_updates[alg])
    plt.scatter(params_obs[0,:], params_obs[1,:], c='xkcd:black', s=10, label='data-generating samples')
    plt.legend()
    plt.xlabel(param_labels[0])
    plt.ylabel(param_labels[1])
    plt.title(f'Color plot of updated density using algorithm {alg+1}')
    plt.colorbar(label='density')
    plt.tight_layout()
    # if alg == 0:
    #     fn = 'plots/wave_joint_4.png'
    # else:
    #     fn = 'plots/wave_joint_4_r2.png'
    # plt.savefig(fn, bbox_inches='tight')

## Quantifying Differences Between DCI solution and True DG Densities Metrics

In [ ]:
# calculating TV metric between updated and exact joint distributions

for alg in [0,1]:
    TV_final = np.abs(pi_updates[alg]-exact_dg)/2
    # TV = np.abs(pi_updates[alg]-kde_dg)/2
    TV_final = np.mean(TV_final)*25
    print(f'TV metric between pi_update and data-generating joint distribution for algorithm {alg+1}: {TV_final}')

In [ ]:
# calculating TVs for marginals at each iteration

marginal_TVs = []
TVs = []
for alg in [0,1]:
    marginal_TVs.append([[],[]])
    for i in range(params.shape[0]):
        # diff = lambda x : np.abs(unif_dist(x,[0.0,5.0])-exact_param_marginals[i](x))
        diff = lambda x : np.abs(unif_dist(x,[0.0,5.0])-kde_param_marginals[i](x))
        TV, _ = quadrature(diff, 0.0, 5.0, tol=1e-2)
        marginal_TVs[alg][i].append(TV/2)
    for i in range(params.shape[0]):
        for j in range(len(r_vals[alg])):
            # diff = lambda x : np.abs(param_marginals[alg][i][j](x)-exact_param_marginals[i](x))
            diff = lambda x : np.abs(param_marginals[alg][i][j](x)-kde_param_marginals[i](x))
            TV, _ = quadrature(diff, 0.0, 5.0, tol=1e-2)
            marginal_TVs[alg][i].append(TV/2)

    TVs.append([])
    pi_init = lambda x, y : unif_dist(x,[0.0,5.0]) * unif_dist(y,[0.0,5.0])
    pi_init = pi_init(params_graphing[0,:], params_graphing[1,:])
    # TV = np.abs(pi_init-exact_dg)/2
    TV = np.abs(pi_init-kde_dg)/2
    TV = np.mean(TV)*25
    TVs[alg].append(TV)
    for j in range(len(r_vals[alg])):
        pi_update = GKDE(params, weights=np.prod(r_vals[alg][:j+1], axis=0))(params_graphing)
        # TV = np.abs(pi_update-exact_dg)/2
        TV = np.abs(pi_update-kde_dg)/2
        TV = np.mean(TV)*25
        TVs[alg].append(TV)

min_marginal_TVs = []
for i in range(params.shape[0]):
    diff = lambda x : np.abs(exact_param_marginals[i](x)-kde_param_marginals[i](x))
    TV, _ = quadrature(diff, 0.0, 5.0, tol=1e-2)
    min_marginal_TVs.append(TV/2)

min_TV = np.abs(kde_dg-exact_dg)/2
min_TV = np.mean(min_TV)*25

In [ ]:
# printing TVs at final iteration

for alg in [0,1]:
    for i in range(params.shape[0]):
        print(f'TV metric for algorithm {alg+1} between final iteration and DG marginals for {param_labels[i]}: {str(marginal_TVs[alg][i][-1])}')

In [ ]:
# plotting TV per iteration

for alg in [0,1]:
    n = range(len(TVs[alg]))
    plt.figure()
    plt.scatter(n,TVs[alg],c=c[0])
    plt.plot(n,TVs[alg],c=c[0],label='TV between updated and exact densities')
    plt.hlines(min_TV,0,len(TVs[alg])-1,linestyles='dashed',colors=c[0],label='TV between exact and KDE of data-generating densities')
    plt.xlabel('Iteration')
    plt.ylabel('TV')
    plt.ylim([0,1])
    plt.title(f'TV metric between updated and exact joint parameter densities for algorithm {alg+1}')
    plt.legend()
    plt.tight_layout()

    for i, t in enumerate(times_used[alg]):
        txt = 't=' + str(t)
        plt.annotate(txt, (n[i+1]+0.1, TVs[alg][i+1]+0.02), size=12)
        
#     if alg == 0:
#         fn = 'plots/wave_TV_joint.png'
#     else:
#         fn = 'plots/wave_TV_joint_r2.png'
#     plt.savefig(fn, bbox_inches='tight')

In [ ]:
# plotting marginal TVs
        
for alg in [0,1]:
    n = range(len(marginal_TVs[alg][0]))
    plt.figure()
    for i in range(params.shape[0]):
        plt.scatter(n,marginal_TVs[alg][i], c=c[i], label='TV between updated and exact marginals for '+param_labels[i])
        plt.plot(n,marginal_TVs[alg][i], c=c[i])
        plt.hlines(min_marginal_TVs[i], 0, len(marginal_TVs[alg][0])-1, linestyles='dashed', colors=c[i],label='TV between exact and KDE of data-generating marginals for '+param_labels[i])
    plt.legend()
    plt.xlabel('iteration')
    plt.ylabel('TV')
    plt.ylim([0,1])
    plt.title(f'TV metric between updated and exact marignal parameter densities for algorithm {alg+1}')
    plt.tight_layout()

    for i, t in enumerate(times_used[alg]):
        txt = 't=' + str(t)
        plt.annotate(txt, (n[i+1]+0.1, np.max(marginal_TVs[alg],axis=0)[i+1]+0.02), size=12)
        
#     if alg == 0:
#         fn = 'plots/wave_TV_marginals.png'
#     else:
#         fn = 'plots/wave_TV_marginals_r2.png'
#     plt.savefig(fn, bbox_inches='tight')